# Inseq：可视化解释LLM的输出

> [HW7: Understand what Generative AI is thinking](https://colab.research.google.com/drive/1Xnz0GHC0yWO2Do0aAYBCq9zL45lbiRjM?usp=sharing#scrollTo=UFOUfh2k1jFN) 中文镜像版
>
> 指导文章：[12. Inseq 特征归因：可视化解释 LLM 的输出](https://github.com/Hoper-J/LLM-Guide-and-Demos-zh_CN/blob/master/Guide/12.%20Inseq%20特征归因：可视化解释%20LLM%20的输出.md)

Feature attribution 特征归因，你可以将其当做对输出的解释，就像图像分类模型可以可视化关注区域一样，LLM 一样也可以。

In [1]:
!uv add inseq
!uv add transformers
!uv add bitsandbytes
!uv add accelerate
!uv add sacremoses
!uv add numpy

Resolved 219 packages in 0.91ms
Audited 204 packages in 0.06ms
Resolved 219 packages in 0.90ms
Audited 204 packages in 0.08ms
Resolved 219 packages in 0.92ms
Audited 204 packages in 0.05ms
Resolved 219 packages in 0.98ms
Audited 204 packages in 0.05ms
Resolved 219 packages in 0.89ms
Audited 204 packages in 0.06ms
Resolved 219 packages in 0.96ms
Audited 204 packages in 0.06ms


## 设置模型下载镜像

注意，需要在导入 inseq 前进行设置才能起效。

In [2]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

## 修复报错：ImportError: cannot import name 'display' from 'IPython.core.display'

当前版本的 inseq 库存在一个导入路径问题。在 `inseq/data/viz.py` 文件中，库尝试从 `IPython.core.display` 导入 `display`，但在新版本的 IPython 中，应该从 `IPython.display` 导入。

执行以下命令修复：

In [3]:
import sys
import IPython.display
import IPython.core
IPython.core.display = IPython.display
sys.modules['IPython.core.display'] = IPython.display

## 可视化翻译任务

In [4]:
import inseq
import torch

print("使用 Helsinki-NLP/opus-mt-zh-en 模型")

# 定义要使用的归因方法列表
attribution_methods = ['saliency', 'attention']

for method in attribution_methods:
    print(f"\n======= 归因方法: {method} =======")

    # 直接用 inseq 加载模型
    inseq_model = inseq.load_model(
        "Helsinki-NLP/opus-mt-zh-en",
        attribution_method=method,
        model_kwargs={
            "attn_implementation": "eager" if method == "attention" else None
        }
    )

    # 准备输入文本
    input_text = "我喜欢机器学习和人工智能。"

    # 进行归因分析
    attribution_result = inseq_model.attribute(
        input_texts=input_text,
        show_progress=True
    )

    # 清理 tokenizer 中的特殊字符（可选）
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('▁', '')
        for item in attr.target:
            item.token = item.token.replace('▁', '')

    # 显示归因结果
    attribution_result.show()

    # 打印生成的翻译
    if attribution_result.sequence_attributions:
        # 获取生成的 tokens
        generated_tokens = attribution_result.sequence_attributions[0].target
        generated_text = " ".join([token.token for token in generated_tokens])
        print(f"翻译结果: {generated_text}")

    # 清理内存
    del inseq_model
    torch.cuda.empty_cache()

使用 Helsinki-NLP/opus-mt-zh-en 模型

======= 归因方法: saliency =======


The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Attributing with saliency...: 100%|██████████| 10/10 [00:00<00:00, 28.06it/s]


,I,like,machine,learning,and,artificial,intelligence,.,</s>
我喜欢,0.24,0.44,0.083,0.113,0.126,0.048,0.041,0.116,0.203
机器,0.159,0.116,0.447,0.178,0.129,0.056,0.059,0.086,0.114
学习,0.074,0.076,0.155,0.283,0.138,0.042,0.048,0.054,0.07
和,0.066,0.055,0.041,0.065,0.179,0.039,0.033,0.048,0.059
人工,0.105,0.073,0.081,0.121,0.134,0.375,0.156,0.158,0.136
智能,0.146,0.104,0.114,0.151,0.171,0.335,0.524,0.407,0.17
。,0.14,0.082,0.035,0.044,0.079,0.054,0.064,0.075,0.17
</s>,0.07,0.053,0.045,0.046,0.044,0.051,0.075,0.056,0.077


翻译结果: <pad> I like machine learning and artificial intelligence . </s>

======= 归因方法: attention =======


The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Attributing with attention...: 2it [00:00, 75.22it/s]               


,I,like,machine,learning,and,artificial,intelligence,.,</s>
我喜欢,0.21,0.537,0.233,0.012,0.026,0.045,0.015,0.042,0.047
机器,0.087,0.033,0.217,0.174,0.043,0.03,0.02,0.017,0.028
学习,0.038,0.038,0.123,0.288,0.142,0.022,0.021,0.029,0.026
和,0.049,0.039,0.056,0.038,0.128,0.093,0.013,0.032,0.028
人工,0.021,0.023,0.037,0.043,0.066,0.254,0.184,0.036,0.032
智能,0.021,0.023,0.063,0.057,0.067,0.113,0.346,0.136,0.027
。,0.135,0.067,0.05,0.029,0.06,0.053,0.043,0.153,0.186
</s>,0.438,0.24,0.221,0.359,0.468,0.391,0.358,0.555,0.625


翻译结果: <pad> I like machine learning and artificial intelligence . </s>


## 可视化文本生成任务

用小模型 Qwen3-0.6B 进行演示，如果你的显存充足，可以替换为更大模型。

In [5]:
import inseq

# 定义要使用的归因方法列表
attribution_methods = ['saliency', 'attention']

for method in attribution_methods:
    print(f"======= 归因方法: {method} =======")

    # 直接用 inseq 加载模型
    inseq_model = inseq.load_model(
        "Qwen/Qwen3-0.6B",
        attribution_method=method,
        model_kwargs={
            "device_map": {"": 0}
        }
    )

    # 设置 padding token
    if inseq_model.tokenizer.pad_token is None:
        inseq_model.tokenizer.pad_token = inseq_model.tokenizer.eos_token
        inseq_model.tokenizer.pad_token_id = inseq_model.tokenizer.eos_token_id

    # 对输入文本进行归因分析
    attribution_result = inseq_model.attribute(
        input_texts="Hello world"
    )

    # 清理 tokenizer 中的特殊字符（可选）
    for attr in attribution_result.sequence_attributions:
        for item in attr.source:
            item.token = item.token.replace('Ġ', '')
        for item in attr.target:
            item.token = item.token.replace('Ġ', '')

    # 显示归因结果
    attribution_result.show()

    # 清理内存
    del inseq_model
    torch.cuda.empty_cache()

======= 归因方法: saliency =======


The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Attributing with saliency...: 100%|██████████| 22/22 [00:02<00:00,  7.47it/s]


,!,This,is,a,simple,and,clean,UI,that,displays,the,current,date,and,time,.,The,UI,has,a
Hello,0.576,0.479,0.342,0.345,0.334,0.25,0.273,0.155,0.253,0.223,0.261,0.246,0.195,0.095,0.118,0.163,0.163,0.184,0.136,0.12
world,0.424,0.324,0.253,0.251,0.246,0.162,0.192,0.113,0.161,0.143,0.201,0.184,0.136,0.061,0.08,0.109,0.119,0.121,0.091,0.082
!,,0.196,0.188,0.138,0.129,0.105,0.062,0.063,0.11,0.093,0.044,0.06,0.058,0.047,0.065,0.072,0.075,0.07,0.065,0.058
This,,,0.216,0.138,0.113,0.105,0.086,0.084,0.062,0.056,0.038,0.045,0.034,0.043,0.04,0.04,0.059,0.049,0.057,0.04
is,,,,0.129,0.091,0.088,0.075,0.065,0.05,0.04,0.031,0.035,0.03,0.038,0.033,0.033,0.043,0.04,0.04,0.034
a,,,,,0.088,0.081,0.077,0.071,0.047,0.042,0.034,0.038,0.032,0.041,0.031,0.03,0.042,0.033,0.04,0.037
simple,,,,,,0.208,0.135,0.097,0.056,0.064,0.04,0.035,0.033,0.047,0.046,0.038,0.059,0.05,0.055,0.056
and,,,,,,,0.1,0.09,0.043,0.062,0.031,0.044,0.03,0.044,0.045,0.026,0.055,0.026,0.039,0.052
clean,,,,,,,,0.263,0.076,0.098,0.051,0.041,0.047,0.067,0.062,0.043,0.069,0.06,0.073,0.062
UI,,,,,,,,,0.141,0.122,0.095,0.084,0.088,0.095,0.08,0.05,0.073,0.138,0.079,0.088


======= 归因方法: attention =======


The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Attributing with attention...: 3it [00:00, 18.40it/s]


,!,This,is,a,sample,program,.,Please,replace,the,sample,program,with,the,correct,one,that,uses,a,class
Hello,0.839,0.739,0.723,0.691,0.674,0.661,0.671,0.608,0.65,0.625,0.611,0.562,0.55,0.583,0.577,0.602,0.556,0.569,0.603,0.637
world,0.161,0.119,0.073,0.063,0.061,0.05,0.032,0.042,0.022,0.033,0.03,0.026,0.014,0.013,0.011,0.012,0.011,0.011,0.014,0.014
!,,0.142,0.092,0.072,0.05,0.044,0.031,0.05,0.031,0.024,0.017,0.017,0.014,0.013,0.01,0.011,0.012,0.011,0.01,0.007
This,,,0.112,0.084,0.056,0.031,0.019,0.031,0.018,0.012,0.011,0.008,0.006,0.008,0.008,0.006,0.005,0.006,0.005,0.005
is,,,,0.09,0.069,0.047,0.023,0.025,0.013,0.009,0.009,0.009,0.005,0.006,0.007,0.005,0.005,0.005,0.004,0.003
a,,,,,0.09,0.049,0.027,0.02,0.011,0.008,0.009,0.008,0.006,0.008,0.008,0.005,0.004,0.005,0.006,0.007
sample,,,,,,0.117,0.081,0.042,0.024,0.024,0.02,0.04,0.014,0.015,0.013,0.013,0.01,0.009,0.008,0.007
program,,,,,,,0.116,0.06,0.033,0.025,0.027,0.036,0.054,0.02,0.021,0.018,0.024,0.014,0.018,0.017
.,,,,,,,,0.12,0.075,0.05,0.035,0.041,0.034,0.032,0.024,0.021,0.022,0.021,0.017,0.014
Please,,,,,,,,,0.122,0.062,0.033,0.022,0.023,0.028,0.021,0.016,0.015,0.015,0.008,0.005
